In [109]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('redshift_config.cfg'))

log_data = config.get('S3', 'LOG_DATA')
song_data = config.get('S3', 'SONG_DATA')

aws_role = config.get('CLUSTER', 'AWS_ROLE')

song_data

"'s3://udacity-dend/song-data/'"

In [98]:
create_staging_events_table = ("""
create table if not exists staging_events(
    artist varchar,
    auth varchar,
    firstName varchar,
    gender varchar,
    itemInSession int,
    lastName varchar,
    length double precision,
    level varchar,
    location varchar,
    method varchar,
    page varchar,
    registration varchar,
    sessionId int, 
    song varchar,
    status int,
    ts bigint,
    userAgent varchar,
    userId varchar
)
"""
)

In [99]:
# varchar(max): to avoid the error while insert data into staging_songs"string length exceeds DDL length"
create_staging_songs_table = ("""
create table if not exists staging_songs(
    song_id varchar,
    num_songs int,
    title varchar(max),
    artist_name varchar(max),
    artist_latitude varchar,
    year int,
    duration double precision,
    artist_id varchar,
    artist_longtitude varchar,
    artist_location varchar(max)
)
"""
)

In [80]:
create_users_dim = ("""
create table if not exists users(
    user_id varchar primary key sortkey,
    first_name varchar,
    last_name varchar,
    gender varchar,
    level varchar
)
"""
)

In [81]:
create_songs_dim = ("""
create table if not exists songs(
    song_id varchar primary key sortkey,
    title varchar(max),
    artist_id varchar REFERENCES artists(artist_id),
    year int,
    duration double precision
)
"""
)

In [82]:
create_artists_dim = ("""
create table if not exists artists(
    artist_id varchar primary key sortkey,
    artist_name varchar(max),
    location varchar(max),
    longtitude varchar,
    latitude varchar,
    num_songs int
)
"""
)

In [83]:
create_time_dim = ("""
create table if not exists time(
    start_time timestamp primary key sortkey,
    year int,
    month int, 
    week int,
    day int,
    hour int,
    is_weekend boolean
)
"""
)

In [84]:
create_songPlay_fact = ("""
create table if not exists songPlay(
    songplay_id int IDENTITY(0,1) SORTKEY,
    start_time timestamp REFERENCES time(start_time),
    artist_id varchar REFERENCES artists(artist_id),
    song_id varchar REFERENCES songs(song_id),
    user_id varchar REFERENCES users(user_id),
    user_level varchar,
    artist_location varchar(max),
    user_agent varchar
)
"""
)

In [112]:
insert_to_staging_events = """
    copy public.staging_events from {0}
    iam_role '{1}'
    format as json 'auto';
""".format(log_data, aws_role)

In [113]:
insert_to_staging_songs = """
    copy public.staging_songs from {0}
    iam_role '{1}'
    format as json 'auto';
""".format(song_data, aws_role)

In [114]:
print(insert_to_staging_events)


    copy public.staging_events from 's3://udacity-dend/log-data/'
    iam_role 'arn:aws:iam::616893511779:role/dwh_role'
    format as json 'auto';



In [115]:
insert_users_dim = """
insert into users
select 
    DISTINCT(userId)
    firstName,
    lastName,
    gender,
    level
from staging_events
where userId is not null
"""

In [116]:
insert_artists_dim = """
insert into artists
select 
    DISTINCT(artist_id),
    artist_name,
    artist_location,
    artist_longtitude, 
    artist_latitude,
    num_songs
from staging_songs
where artist_id is not null
"""

In [117]:
insert_songs_dim = """
insert into songs
select 
    DISTINCT(song_id),
    title,
    artist_id,
    year,
    duration
from staging_songs
where song_id is not null
"""

Links to know more about epoch timestamp, how to convert from epoch to date and use date_part:
- https://www.epochconverter.com/
- https://www.fernandomc.com/posts/redshift-epochs-and-timestamps/
- https://docs.aws.amazon.com/redshift/latest/dg/r_DATE_PART_function.html

In [118]:
#The day of week date part returns an integer from 0-6, starting with Sunday. 5,6 are friday and saturday
insert_time_dim = """
insert into time
select
   (timestamp 'epoch' + ts/1000 * interval '1 second')::timestamp AS start_time,
   DATE_PART(year, start_time),
   DATE_PART(month, start_time),
   DATE_PART(week, start_time),
   DATE_PART(day, start_time),
   DATE_PART(hour, start_time),
   case when DATE_PART(dow, start_time) in (5,6) then TRUE else FALSE 
   END
from staging_events
"""

In [119]:
insert_songPlay_fact = """
insert into songPlay(start_time, artist_id, song_id, user_id, user_level, artist_location, user_agent)
select
    DISTINCT((timestamp 'epoch' + ts/1000 * interval '1 second')::timestamp) AS start_time,
    so.artist_id,
    so.song_id,
    se.userId,
    se.level,
    so.artist_location,
    se.userAgent
from staging_events se
join staging_songs so
on se.artist = so.artist_name
and se.song = so.title
where se.page = 'NextSong'
"""

In [120]:
drop_staging_events = "Drop table if exists staging_events"
drop_staging_songs = "Drop table if exists staging_songs"
drop_time_dimention = "Drop table if exists time CASCADE "
drop_users_dimention = "Drop table if exists users CASCADE "
drop_artist_dimention = "Drop table if exists artists CASCADE "
drop_song_dimention = "Drop table if exists songs CASCADE "
drop_songPlay_fact = "Drop table if exists songPlay"

In [121]:
trunc_staging_events = "truncate table staging_events"
trunc_staging_songs = "truncate table staging_songs"

In [122]:
trunc_users_dim= "truncate table users"
trunc_songs_dim= "truncate table songs"
trunc_artists_dim= "truncate table artists"
trunc_time_dim= "truncate table time"

In [123]:
trunc_dim_tables = [trunc_users_dim, trunc_songs_dim, trunc_artists_dim, trunc_time_dim]

In [124]:
drop_staging_tables = [drop_staging_events, drop_staging_songs]

In [125]:
drop_dimFact_tables = [drop_time_dimention, drop_users_dimention, drop_artist_dimention, drop_song_dimention, drop_songPlay_fact]

In [126]:
create_staging_tables = [create_staging_events_table, create_staging_songs_table]

In [127]:
trunc_staging_tables = [trunc_staging_events, trunc_staging_songs]

In [128]:
insert_staging_tables = [insert_to_staging_events, insert_to_staging_songs]

In [129]:
create_dim_facts_tables = [create_users_dim, create_songs_dim, create_artists_dim, create_time_dim, create_songPlay_fact]

In [130]:
insert_dim_tables = [insert_users_dim, insert_artists_dim, insert_songs_dim, insert_time_dim]

In [131]:
insert_fact_table = [insert_songPlay_fact]

- The %store magic command saves the specified variable and lets you pass the value of variables between two notebooks. Here’s how we can pass the variable value

- If you notice, we deleted the variables inside the notebook after we stored them in %store . We were able to derive them in the second notebook because we never deleted them from %store

In [132]:
%store drop_staging_tables
del drop_staging_tables

Stored 'drop_staging_tables' (list)


In [133]:
%store drop_dimFact_tables
del drop_dimFact_tables

Stored 'drop_dimFact_tables' (list)


In [134]:
%store create_staging_tables
del create_staging_tables

Stored 'create_staging_tables' (list)


In [135]:
%store create_dim_facts_tables
del create_dim_facts_tables

Stored 'create_dim_facts_tables' (list)


In [136]:
%store trunc_staging_tables
del trunc_staging_tables

Stored 'trunc_staging_tables' (list)


In [137]:
%store insert_staging_tables
del insert_staging_tables

Stored 'insert_staging_tables' (list)


In [138]:
%store trunc_dim_tables
del trunc_dim_tables

Stored 'trunc_dim_tables' (list)


In [139]:
%store insert_dim_tables
del insert_dim_tables

Stored 'insert_dim_tables' (list)


In [140]:
%store insert_fact_table
del insert_fact_table

Stored 'insert_fact_table' (list)
